## 1. 크롤링과 데이터베이스 예제

* cine21 배우 랭킹 사이트 크롤링
    - 사이트 주소 : http://www.cine21.com/rank/person/
    - 요청 방식 확인 방법 : 크롬 개발자 모드로 들어가서, Network -> content 페이지의 요청 방식 확인
        - Requests Url: http://www.cine21.com/rank/person/content
        - Requests Method : POST
        - Form data
            - section = 'actor'
            - period_start: 2020-11
            - gender: all
            - page: 1

1.  라이브러리 import

In [7]:
from bs4 import BeautifulSoup
import requests
import pymongo
import re

### 2. mongodb connection

In [ ]:
conn = pymongo.MongoClient()
actor_db = conn['cine21']
actor_collection = actor_db['actor_collection']


#### 3. 크롤링 주소 requests

In [31]:
url = 'http://www.cine21.com/rank/person/content'
post_data = dict()
post_data['section'] = 'actor'
post_data['period_start'] = '2020-11'
post_data['gender'] = 'all'
post_data['page'] = 1

res = requests.post(url, data=post_data)

### 4. parsing  과 데이터 추출

In [32]:
soup = BeautifulSoup(res.content, 'html.parser')

In [33]:
actors = soup.select("li.people_li  div.name")
for actor in actors:
#     print(actor.get_text().split("(")[0])
    print(re.sub('\([A-Za-z0-9가-힣]*\)','',actor.get_text()))

이제훈
조우진
신혜선
한예리
김영광
윤여정
임원희


In [26]:
test_data = '주지훈(6편)'
re.sub('\([A-Za-z0-9가-힣]*\)','',test_data)

'주지훈'

### 5. 배우 상세정보 추출

In [46]:
actors = soup.select("li.people_li  div.name")
for actor in actors:
    actor_link = 'http://www.cine21.com' + actor.select_one("a").attrs['href']
    response_actor = requests.get(actor_link)
    soup_actor = BeautifulSoup(response_actor.content, 'html.parser')
    default_info = soup_actor.select_one("ul.default_info")
    actor_details = default_info.select('li')
    for actor_item in actor_details:
        print(actor_item)

<li><span class="tit">직업</span>배우</li>
<li><span class="tit">생년월일</span>1984-07-04</li>
<li><span class="tit">성별</span>남</li>
<li><span class="tit">홈페이지</span>
<a href="https://www.facebook.com/saram.leejehoon" target="_blank">https://www.facebook.com/saram.leejehoon</a><br/>
</li>
<li><span class="tit">신장/체중</span>176cm</li>
<li><span class="tit">학교</span>한국예술종합학교 연극원 연기과</li>
<li><span class="tit">특기</span>댄스, 노래, 무술</li>
<li><span class="tit">소속사</span>㈜사람엔터테인먼트</li>
<li><span class="tit">다른 이름</span>조신제</li>
<li><span class="tit">직업</span>배우</li>
<li><span class="tit">생년월일</span>1979-01-16</li>
<li><span class="tit">성별</span>남</li>
<li><span class="tit">직업</span>배우</li>
<li><span class="tit">생년월일</span>1989-00-00</li>
<li><span class="tit">성별</span>여</li>
<li><span class="tit">홈페이지</span>
<a href="https://www.facebook.com/shs831" target="_blank">https://www.facebook.com/shs831</a><br/>
<a href=" https://www.instagram.com/shinhs831/" target="_blank"> https://www.instagram.com/shinhs